In [22]:
import torch
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
import wandb

In [ ]:
run = wandb.init(project="Head Word Final 2")

In [ ]:
api = wandb.Api()

In [ ]:
runs = {}
for run in api.runs(
    path="ananthag/Head Word Final 2"
):
    runs[run.config['model_name']] = run.id

In [36]:
tok.convert_tokens_to_ids('bite')

37018

In [5]:
tok.decode(4532)

CodeGenTokenizerFast(name_or_path='microsoft/phi-2', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50257: AddedToken("                               ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50258: AddedToken("                              ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50259: AddedToken("                             ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50260: AddedToken("                            ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50261: AddedToken("            

In [13]:
# tok.encode(' adjust', add_prefix_space=False)
type(tok)

transformers.models.codegen.tokenization_codegen_fast.CodeGenTokenizerFast

In [23]:
not tok.pad_token_id

True

In [14]:
inputs = ["Hey what's your name", "Welcome home", "text" * 20]
tok.pad_token_id = tok.eos_token_id
inputs_dict = tok.batch_encode_plus(inputs, padding=True, return_tensors='pt')
from transformers import AutoModelForCausalLM

m = AutoModelForCausalLM.from_pretrained('gpt2')
out = m(**inputs_dict)

In [20]:
tok.pad_token_id

50256

In [83]:
inputs = inputs_dict["input_ids"]

In [57]:
attn_mask = inputs_dict["attention_mask"]

last_non_masked_idx = torch.sum(attn_mask, dim=1) - 1

In [49]:
position_ids = torch.tensor(
            [list(range(inputs.shape[1])) for i in range(inputs.shape[0])]
        )

In [61]:
for i, position_ids_slice in enumerate(position_ids):
    position_ids_slice[last_non_masked_idx[i] :] = position_ids_slice[
        last_non_masked_idx[i]
    ]

In [50]:
torch.arange(inputs.shape[1]).repeat(inputs.shape[0], 1)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19]])

In [64]:
inputs[attn_mask] = 2

In [66]:
attn_mask

tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [80]:

inputs.scatter_(0, 1 - attn_mask, last_non_masked_idx.unsqueeze(1))
inputs


RuntimeError: Expected index [3, 20] to be smaller than self [3, 20] apart from dimension 0 and to be smaller size than src [3, 1]

In [98]:
inputs = inputs * attn_mask + (~attn_mask.bool() * last_non_masked_idx.unsqueeze(1).expand(-1, inputs.shape[1]))

tensor([[False, False, False, False, False,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False]])

In [99]:
inputs

tensor([[10814,   644,   338,   534,  1438,     4,     4,     4,     4,     4,
             4,     4,     4,     4,     4,     4,     4,     4,     4,     4],
        [14618,  1363,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [ 5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,
          5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239]])

In [2]:
import dataset
from dataset import DepParseDataPickle

d = dataset.HeadWordDatasetWithRelns('test', 'gpt2', 12, 768)

In [9]:
len(d.start_indices)

2416

In [10]:
d.start_indices[2415]

58223

In [12]:
d.hidden_state_cache.shape

(58236, 12, 768)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
X = [[0, 0, 0], [1, 1, 1]]
Y = [[1, 0, 0], [1, 1, 0]]
cosine_similarity(Y, Y)

array([[1.        , 0.70710678],
       [0.70710678, 1.        ]])

In [19]:
import numpy as np

ran = np.random.rand(3, 3)
ran[np.eye(3, 3, dtype=bool)] = 0

In [23]:
ran.argmax(axis=1)

array([3, 3, 2])

In [16]:
import csv
import collections
counter = collections.Counter()
models = set()
with open('surprisals_errata.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        counter[(row['file_name'], row['item_number'])] += 1
        models.add(row['model'])

len(models)        

15

In [15]:
counter.most_common(100)

[(('reflexive_src_fem.json', '6'), 15),
 (('reflexive_src_fem.json', '7'), 15),
 (('reflexive_src_fem.json', '13'), 15),
 (('reflexive_src_fem.json', '16'), 15),
 (('reflexive_prep_fem.json', '1'), 15),
 (('reflexive_prep_fem.json', '2'), 15),
 (('reflexive_src_fem.json', '1'), 14),
 (('reflexive_src_fem.json', '5'), 14),
 (('reflexive_src_fem.json', '15'), 14),
 (('reflexive_src_fem.json', '19'), 14),
 (('reflexive_prep_fem.json', '17'), 14),
 (('fgd-embed3.json', '4'), 14),
 (('fgd_subject.json', '4'), 14),
 (('fgd_subject.json', '5'), 14),
 (('fgd_subject.json', '9'), 14),
 (('fgd_subject.json', '10'), 14),
 (('mvrr.json', '5'), 14),
 (('mvrr_mod.json', '5'), 14),
 (('number_orc.json', '5'), 14),
 (('number_orc.json', '17'), 14),
 (('number_prep.json', '5'), 14),
 (('number_prep.json', '17'), 14),
 (('number_src.json', '5'), 14),
 (('number_src.json', '17'), 14),
 (('reflexive_orc_fem.json', '1'), 14),
 (('reflexive_orc_fem.json', '5'), 14),
 (('reflexive_orc_fem.json', '6'), 14),
 

In [1]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
    GPTNeoXTokenizerFast,
    LlamaTokenizer,
    GPT2Tokenizer,
    GemmaTokenizer,
)


In [3]:
batch = ["cat", "dog", "cow", "bird"]
model_name = 'google/gemma-2b'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast='pythia' in 'google')
# tokenizer.pad_token_id = tokenizer.eos_token_id
# batch = tokenizer(("one word", "two two two words", "three three three three three words"), padding=True, return_tensors='pt')

# model = AutoModelForCausalLM.from_pretrained('gpt2-large')
# output = model(**batch)
tokenizer

GemmaTokenizerFast(name_or_path='google/gemma-2b', vocab_size=256000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<bos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [17]:
batch

{'input_ids': tensor([[  505,  1573, 50256, 50256, 50256, 50256],
        [11545,   734,   734,  2456, 50256, 50256],
        [15542,  1115,  1115,  1115,  1115,  2456]]), 'attention_mask': tensor([[1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1]])}

In [18]:
batch["input_ids"]

tensor([[  505,  1573, 50256, 50256, 50256, 50256],
        [11545,   734,   734,  2456, 50256, 50256],
        [15542,  1115,  1115,  1115,  1115,  2456]])

In [14]:
batch.attention_mask.unsqueeze(-1)

tensor([[[1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [0]],

        [[1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1]]])

In [24]:
import torch.nn.functional as F
targets = torch.cat((batch["input_ids"][:, 1:], torch.tensor([tokenizer.eos_token_id]).expand(batch["input_ids"].shape[0], 1)), dim=1)
out = torch.gather(F.log_softmax(output.logits, dim=-1), -1, targets.unsqueeze(-1))
(out * batch.attention_mask.unsqueeze(-1)).view(batch.input_ids.shape[0], -1).sum(-1)

tensor([-17.2175, -27.2265, -33.0799], grad_fn=<SumBackward1>)

In [26]:
import os
os.listdir("../blimp/data")

['animate_subject_passive.jsonl',
 'principle_A_domain_3.jsonl',
 'npi_present_1.jsonl',
 'wh_vs_that_no_gap_long_distance.jsonl',
 'existential_there_quantifiers_1.jsonl',
 'determiner_noun_agreement_irregular_1.jsonl',
 'ellipsis_n_bar_1.jsonl',
 'causative.jsonl',
 'wh_questions_object_gap.jsonl',
 'transitive.jsonl',
 'superlative_quantifiers_1.jsonl',
 'tough_vs_raising_2.jsonl',
 'principle_A_case_1.jsonl',
 'expletive_it_object_raising.jsonl',
 'irregular_plural_subject_verb_agreement_2.jsonl',
 'principle_A_domain_2.jsonl',
 'sentential_negation_npi_scope.jsonl',
 'ellipsis_n_bar_2.jsonl',
 'determiner_noun_agreement_irregular_2.jsonl',
 'left_branch_island_echo_question.jsonl',
 'existential_there_quantifiers_2.jsonl',
 'wh_island.jsonl',
 'superlative_quantifiers_2.jsonl',
 'principle_A_domain_1.jsonl',
 'irregular_plural_subject_verb_agreement_1.jsonl',
 'distractor_agreement_relative_clause.jsonl',
 'existential_there_subject_raising.jsonl',
 'tough_vs_raising_1.jsonl',
 'p

In [34]:
targets = torch.cat(
    (
        batch.input_ids[:, 1:],
        torch.tensor([[tokenizer.eos_token_id]]).expand(
            batch.input_ids.shape[0], 1
        ),
    ),
    dim=-1,
)
logprobs = torch.gather(
    torch.nn.functional.log_softmax(output.logits, dim=-1),
    dim=-1,
    index=targets.unsqueeze(1),
).reshape(-1)
logprobs

tensor([-6.4905, -6.7212, -6.6658, -6.3376], grad_fn=<ViewBackward0>)

In [37]:
torch.sum(logprobs > logprobs)

tensor(0)

In [21]:
torch.tensor([[tokenizer.eos_token_id]]).expand(4, 1)

tensor([[50256],
        [50256],
        [50256],
        [50256]])

In [3]:
AutoModelForCausalLM.from_pretrained('google/gemma-2b').config

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GemmaConfig {
  "_name_or_path": "google/gemma-2b",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 16384,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 256000
}

In [1]:
import transformers
transformers.__version__

'4.41.0'

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
olmo = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B-hf")
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-hf")

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.71G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-hf")

In [7]:
tokenizer("Hello world")

{'input_ids': [12092, 1533], 'attention_mask': [1, 1]}

In [10]:
tokenizer(" world")

{'input_ids': [1533], 'attention_mask': [1]}

In [33]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
import numpy as np
import torch.nn.functional as F

def compute_pseudoperplexity(model, tokenizer, text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors='pt')
    input_ids = inputs['input_ids']
    print(input_ids.shape)
    attention_mask = inputs['attention_mask']
    
    # Get the length of the sequence
    seq_length = input_ids.size(1)
    
    # Create batched inputs with each token masked once
    masked_input_ids = input_ids.repeat(seq_length, 1)
    
    masked_attention_mask = attention_mask.repeat(seq_length, 1)
    masked_input_ids.fill_diagonal_(tokenizer.mask_token_id)
    
    # Get the model's predictions for the batched masked inputs
    with torch.inference_mode():
        outputs = model(masked_input_ids, attention_mask=masked_attention_mask)
    
    # Extract the logits and compute the log softmax to get log probabilities
    logits = outputs.logits
    log_probs = F.log_softmax(logits, dim=-1)
    
    # Get the actual token IDs
    actual_token_ids = input_ids[0]

    # Extract the log probabilities of the actual tokens
    token_log_probs = log_probs[torch.arange(seq_length), torch.arange(seq_length), actual_token_ids]
    
    # Compute the sum of log probabilities
    return token_log_probs.sum().item()

# Load pre-trained BERT model and tokenizer
model_name = 'FacebookAI/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Example text
text = "The quick brown fox jumped."

# Compute pseudoperplexity
print(f"Pseudoperplexity: {compute_pseudoperplexity(model, tokenizer, 'He is cool.')}")
print(f"Pseudoperplexity: {compute_pseudoperplexity(model, tokenizer, 'He are cool.')}")


torch.Size([1, 6])
Pseudoperplexity: -18.2569522857666
torch.Size([1, 6])
Pseudoperplexity: -30.995452880859375


In [6]:
from transformers import AutoModelForMaskedLM, DebertaV2ForMaskedLM

model = AutoModelForMaskedLM.from_pretrained('microsoft/deberta-base')

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
for k in model.state_dict().keys():
    print(k)

deberta.embeddings.word_embeddings.weight
deberta.embeddings.LayerNorm.weight
deberta.embeddings.LayerNorm.bias
deberta.encoder.layer.0.attention.self.q_bias
deberta.encoder.layer.0.attention.self.v_bias
deberta.encoder.layer.0.attention.self.in_proj.weight
deberta.encoder.layer.0.attention.self.pos_proj.weight
deberta.encoder.layer.0.attention.self.pos_q_proj.weight
deberta.encoder.layer.0.attention.self.pos_q_proj.bias
deberta.encoder.layer.0.attention.output.dense.weight
deberta.encoder.layer.0.attention.output.dense.bias
deberta.encoder.layer.0.attention.output.LayerNorm.weight
deberta.encoder.layer.0.attention.output.LayerNorm.bias
deberta.encoder.layer.0.intermediate.dense.weight
deberta.encoder.layer.0.intermediate.dense.bias
deberta.encoder.layer.0.output.dense.weight
deberta.encoder.layer.0.output.dense.bias
deberta.encoder.layer.0.output.LayerNorm.weight
deberta.encoder.layer.0.output.LayerNorm.bias
deberta.encoder.layer.1.attention.self.q_bias
deberta.encoder.layer.1.attenti

In [14]:
!echo $HF_HOME

/scr/biggest/ananthag


In [3]:
from transformers import AutoModel, AutoModelForMaskedLM

roberta = AutoModelForMaskedLM.from_pretrained('FacebookAI/roberta-base')


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [5]:
roberta.lm_head.decoder.weight

Parameter containing:
tensor([[ 0.1476, -0.0365,  0.0753,  ..., -0.0023,  0.0172, -0.0016],
        [ 0.0156,  0.0076, -0.0118,  ..., -0.0022,  0.0081, -0.0156],
        [-0.0347, -0.0873, -0.0180,  ...,  0.1174, -0.0098, -0.0355],
        ...,
        [ 0.0304,  0.0504, -0.0307,  ...,  0.0377,  0.0096,  0.0084],
        [ 0.0623, -0.0596,  0.0307,  ..., -0.0920,  0.1080, -0.0183],
        [ 0.1259, -0.0145,  0.0332,  ...,  0.0121,  0.0342,  0.0168]],
       requires_grad=True)

In [16]:
import torch

state_dict = torch.load('/scr/ananthag/hub/models--microsoft--deberta-v3-xsmall/snapshots/4b419818330868dff6a60ad3e6b1c730f8b8c0c6/pytorch_model.bin')

In [18]:
for k, v in state_dict.items():
    print(k, v.shape)

deberta.embeddings.word_embeddings._weight torch.Size([128100, 384])
deberta.embeddings.word_embeddings.weight torch.Size([128100, 384])
deberta.embeddings.position_embeddings._weight torch.Size([512, 384])
deberta.embeddings.position_embeddings.weight torch.Size([512, 384])
deberta.embeddings.LayerNorm.weight torch.Size([384])
deberta.embeddings.LayerNorm.bias torch.Size([384])
deberta.encoder.layer.0.attention.self.query_proj.weight torch.Size([384, 384])
deberta.encoder.layer.0.attention.self.query_proj.bias torch.Size([384])
deberta.encoder.layer.0.attention.self.key_proj.weight torch.Size([384, 384])
deberta.encoder.layer.0.attention.self.key_proj.bias torch.Size([384])
deberta.encoder.layer.0.attention.self.value_proj.weight torch.Size([384, 384])
deberta.encoder.layer.0.attention.self.value_proj.bias torch.Size([384])
deberta.encoder.layer.0.attention.output.dense.weight torch.Size([384, 384])
deberta.encoder.layer.0.attention.output.dense.bias torch.Size([384])
deberta.encoder.

In [23]:
for k, v in deberta.state_dict().items():
    print(k, v.shape)

deberta.embeddings.word_embeddings.weight torch.Size([128100, 384])
deberta.embeddings.LayerNorm.weight torch.Size([384])
deberta.embeddings.LayerNorm.bias torch.Size([384])
deberta.encoder.layer.0.attention.self.query_proj.weight torch.Size([384, 384])
deberta.encoder.layer.0.attention.self.query_proj.bias torch.Size([384])
deberta.encoder.layer.0.attention.self.key_proj.weight torch.Size([384, 384])
deberta.encoder.layer.0.attention.self.key_proj.bias torch.Size([384])
deberta.encoder.layer.0.attention.self.value_proj.weight torch.Size([384, 384])
deberta.encoder.layer.0.attention.self.value_proj.bias torch.Size([384])
deberta.encoder.layer.0.attention.output.dense.weight torch.Size([384, 384])
deberta.encoder.layer.0.attention.output.dense.bias torch.Size([384])
deberta.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([384])
deberta.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([384])
deberta.encoder.layer.0.intermediate.dense.weight torch.Size([1536, 384])


In [ ]:
['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']


In [39]:
for name, param in model.cls.named_parameters():
    print(name, param.shape)

predictions.bias torch.Size([128100])
predictions.transform.dense.weight torch.Size([384, 384])
predictions.transform.dense.bias torch.Size([384])
predictions.transform.LayerNorm.weight torch.Size([384])
predictions.transform.LayerNorm.bias torch.Size([384])
predictions.decoder.weight torch.Size([128100, 384])


In [44]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [36]:
tokenizer.mask_token_id

In [58]:
inputs = tokenizer("Hello <extra_id_0>", return_tensors='pt', padding=True)
labels = tokenizer("this is <extra_id_0> world", return_tensors='pt', padding=True).input_ids
output = model(**inputs, labels=labels)

In [66]:
tokenizer.decode(32099)

'<extra_id_0>'

In [60]:
output.logits.shape

torch.Size([1, 5, 32128])

In [57]:
output

Seq2SeqLMOutput(loss=tensor(6.2130, grad_fn=<NllLossBackward0>), logits=tensor([[[-43.1238, -12.7323, -26.8814,  ..., -66.6852, -66.6937, -66.6066],
         [-27.5227,  -5.1289,  -6.9508,  ..., -41.1360, -41.1700, -41.1575],
         [-28.0383,  -6.3891,  -5.8660,  ..., -40.2829, -40.2964, -40.3289]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-0.4865, -2.3323, -1.1428,  ..., -2.5693, -1.7539, -0.5693],
          [-0.6358, -1.1153, -0.3137,  ..., -0.7512, -0.5316, -0.5639],
          [ 1.6587,  3.2717, -2.2787,  ...,  0.3732, -1.0274,  0.8426]],

         [[-0.9565, -2.3581, -0.2478,  ...,  1.1193,  0.7770,  0.5377],
          [ 2.1201, -1.6266, -0.7735,  ...,  1.0306,  1.5982,  6.4674],
          [ 1.4973, -0.2845, -0.7443,  ...,  0.9890,  1.3783, -1.7426]],

         [[ 1.0704, -2.5709,  0.6892,  ..., -1.3349,  0.2768,  0.5756],
          [ 0.8899, -3.0563,  0.7858,  ...,  0.4327, -0.8198,  1.0157],
          [-0.1107, -0.7388,  0.5365,  ...,  1.1146,  0.75

In [29]:
input_text = "The quick brown fox"
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small", use_fast=False)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

In [76]:
input_text = "He are cool"
tokenized = tokenizer(input_text, return_tensors="pt")
input_ids = tokenized.input_ids
attention_mask = tokenized.attention_mask
seq_length = input_ids.shape[1]

# Mask out each token in the input sequence
masked_input_ids = input_ids.repeat(seq_length, 1)
masked_attention_mask = attention_mask.repeat(seq_length, 1)
masked_input_ids.fill_diagonal_(mask_token)
masked_input_ids


tensor([[32099,    33,  1633,     1],
        [  216, 32099,  1633,     1],
        [  216,    33, 32099,     1],
        [  216,    33,  1633, 32099]])

In [77]:
mask_token = tokenizer.convert_tokens_to_ids(tokenizer.additional_special_tokens[0])
mask_token

32099

In [78]:
decoder_input_ids = masked_input_ids.tril()

In [79]:
import torch
decoder_inputs = torch.tensor([[model.config.decoder_start_token_id, mask_token]]).expand(seq_length, 2)

In [80]:
logits = model(input_ids=masked_input_ids, attention_mask=masked_attention_mask, decoder_input_ids=decoder_inputs).logits

In [81]:
logits[torch.arange(seq_length), 1, input_ids.squeeze()].sum()

tensor(-38.9922, grad_fn=<SumBackward0>)

In [94]:
tokenizer.batch_decode(logits.argmax(-1))

['cool guys', '<extra_id_0> is', '<extra_id_0>.', 'cool cool']

In [93]:
tokenizer.batch_decode(masked_input_ids)

['<extra_id_0> are cool</s>',
 'He <extra_id_0> cool</s>',
 'He are <extra_id_0> </s>',
 'He are cool <extra_id_0>']

In [97]:
tokenizer("He are cool.")

{'input_ids': [216, 33, 1633, 5, 1], 'attention_mask': [1, 1, 1, 1, 1]}

In [252]:
from transformers import AutoTokenizer, RobertaForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base", use_fast=False)
model = RobertaForMaskedLM.from_pretrained("FacebookAI/roberta-base")

In [254]:
inputs = tokenizer("The capital of France is <mask>.", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of <mask>
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

# labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# mask labels of non-<mask> tokens
# labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

# outputs = model(**inputs, labels=labels)
# round(outputs.loss.item(), 2)

' Paris'

In [256]:
tokenizer.decode(torch.topk(logits[0, mask_token_index], 3, -1).indices[0])

' Paris Lyon Nice'

In [8]:
logits[0, 6].argmax(axis=-1)

tensor(2201)

In [9]:
predicted_token_id

tensor([2201])

In [2]:
tokenizer

RobertaTokenizerFast(name_or_path='FacebookAI/roberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [13]:
tokenizer("Hello world")

{'input_ids': [0, 31414, 232, 2], 'attention_mask': [1, 1, 1, 1]}

In [20]:
tokenizer.

'<s>'

In [23]:
gemma = AutoTokenizer.from_pretrained("google/gemma-2b", use_fast=False)

In [30]:
tokenizer.eos_token_id

2

In [36]:
import numpy as np
import torch

t = torch.rand(5)


2.130485951900482

In [42]:
torch.sum(t, dim=0)

tensor(2.1305)

In [257]:
from DeBERTa.DeBERTa.apps.models import masked_language_model
from DeBERTa.DeBERTa.deberta import config
import pkgutil
from huggingface_hub import hf_hub_download

config_json = hf_hub_download('microsoft/deberta-v3-base', 'config.json')
# model = (config.ModelConfig.from_json_file(config_json))
model = masked_language_model.MaskedLanguageModel.load_model("deberta-v3-base")

05/23/2024 00:57:09|INFO|logging|00| Loaded pretrained model file /afs/cs.stanford.edu/u/ananthag/.~DeBERTa/assets/latest/deberta-v3-base/pytorch_model.bin
05/23/2024 00:57:13|WARNING|logging|00| Missing keys: [], unexpected_keys: ['mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias'], error_msgs: []


In [248]:
model.config.position_biased_input = True

In [202]:
import torch
from huggingface_hub import hf_hub_download
checkpoint_path = hf_hub_download('microsoft/deberta-v3-base', 'pytorch_model.bin')
ckpt = torch.load(checkpoint_path)

In [203]:
model.load_state_dict(ckpt, strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias'])

In [53]:
import torch
from huggingface_hub import hf_hub_download
checkpoint_path = hf_hub_download('microsoft/deberta-v3-xsmall', 'pytorch_model.bin')
ckpt = torch.load(checkpoint_path)
ckpt['cls.predictions.transform.dense.weight'] = ckpt.pop('lm_predictions.lm_head.dense.weight')
ckpt['cls.predictions.transform.dense.bias'] = ckpt.pop('lm_predictions.lm_head.dense.bias')
ckpt['cls.predictions.transform.LayerNorm.weight'] = ckpt.pop('lm_predictions.lm_head.LayerNorm.weight')
ckpt['cls.predictions.transform.LayerNorm.bias'] = ckpt.pop('lm_predictions.lm_head.LayerNorm.bias')
ckpt['cls.predictions.decoder.weight'] = ckpt['deberta.embeddings.word_embeddings.weight']
ckpt['cls.predictions.decoder.bias'] = ckpt['lm_predictions.lm_head.bias']
ckpt['cls.predictions.bias'] = ckpt.pop('lm_predictions.lm_head.bias')

In [76]:
from transformers import AutoModel

AutoModel.from_pretrained("microsoft/deberta-v3-xsmall").config
config_json = hf_hub_download("microsoft/deberta-v3-xsmall", "config.json")
from DeBERTa.DeBERTa.deberta import config
c = config.ModelConfig.from_json_file(config_json)

In [79]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall")
# tokenizer("Hi this")
c.position_biased_input = True

In [267]:
from DeBERTa.DeBERTa import deberta
# from DeBERTa.DeBERTa.apps.tasks import mlm_task
vocab_path, vocab_type = deberta.load_vocab(pretrained_id='deberta-v3-base')
tokenizer = deberta.tokenizers[vocab_type](vocab_path)
# We apply the same schema of special tokens as BERT, e.g. [CLS], [SEP], [MASK]
max_seq_len = 512
tokens = tokenizer.tokenize('The capital of France is ')
tokens


['▁The', '▁capital', '▁of', '▁France', '▁is']

In [268]:
# tokens = tokenizer.tokenize('A B C D E F')
tokens.append(tokenizer.mask())
token_ids = ['[CLS]'] + tokens + ['[SEP]']
input_ids = tokenizer.convert_tokens_to_ids(token_ids)
input_ids

[1, 279, 1909, 265, 2378, 269, 128000, 2]

In [269]:
logits = model(
    torch.tensor([input_ids]),
    position_ids = torch.arange(len(input_ids)).unsqueeze(0),
    input_mask=torch.ones(len(input_ids)).unsqueeze(0),
    labels=torch.tensor([[0, 0, 0, 0, 0, 0, 3045, 0]])
)['logits']

In [251]:
input_ids

[1, 1336, 269, 262, 128000, 265, 1991, 2]

In [270]:
hf_tokenizer.decode(torch.topk(logits, 10, dim=-1).indices[0])

'akshshenkoarnaArabPINK bitmap fraying Kumasi NAIanthi'

In [200]:
model.lm_predictions.lm_head.dense.bias

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.

In [198]:
tokenizer.decode(['65618'])

'65618'

In [85]:
from bisect import bisect
import math

class NGramMaskGenerator:
  """
  Mask ngram tokens
  https://github.com/zihangdai/xlnet/blob/0b642d14dd8aec7f1e1ecbf7d6942d5faa6be1f0/data_utils.py
  """
  def __init__(self, tokenizer, mask_lm_prob=0.15, max_seq_len=512, max_preds_per_seq=None, max_gram = 1, keep_prob = 0.1, mask_prob=0.8, **kwargs):
    self.tokenizer = tokenizer
    self.mask_lm_prob = mask_lm_prob
    self.keep_prob = keep_prob
    self.mask_prob = mask_prob
    assert self.mask_prob+self.keep_prob<=1, f'The prob of using [MASK]({mask_prob}) and the prob of using original token({keep_prob}) should between [0,1]'
    self.max_preds_per_seq = max_preds_per_seq
    if max_preds_per_seq is None:
      self.max_preds_per_seq = math.ceil(max_seq_len*mask_lm_prob /10)*10

    self.max_gram = max(max_gram, 1)
    self.mask_window = int(1/mask_lm_prob) # make ngrams per window sized context
    self.vocab_words = list(tokenizer.vocab.keys())

  def mask_tokens(self, tokens, rng, **kwargs):
    special_tokens = ['[MASK]', '[CLS]', '[SEP]', '[PAD]', '[UNK]'] # + self.tokenizer.tokenize(' ')
    indices = [i for i in range(len(tokens)) if tokens[i] not in special_tokens]
    ngrams = np.arange(1, self.max_gram + 1, dtype=np.int64)
    pvals = 1. / np.arange(1, self.max_gram + 1)
    pvals /= pvals.sum(keepdims=True)

    unigrams = []
    for id in indices:
      if self.max_gram>1 and len(unigrams)>=1 and self.tokenizer.part_of_whole_word(tokens[id]):
        unigrams[-1].append(id)
      else:
        unigrams.append([id])
    
    num_to_predict = min(self.max_preds_per_seq, max(1, int(round(len(tokens) * self.mask_lm_prob))))
    mask_len = 0
    offset = 0
    mask_grams = np.array([False]*len(unigrams))
    while offset < len(unigrams):
      n = self._choice(rng, ngrams, p=pvals)
      ctx_size = min(n*self.mask_window, len(unigrams)-offset)
      m = rng.randint(0, ctx_size-1)
      s = offset + m
      e = min(offset+m+n, len(unigrams))
      offset = max(offset+ctx_size, e)
      mask_grams[s:e] = True

    target_labels = [None]*len(tokens)
    w_cnt = 0
    for m,word in zip(mask_grams, unigrams):
      if m:
        for idx in word:
          label = self._mask_token(idx, tokens, rng, self.mask_prob, self.keep_prob)
          target_labels[idx] = label
          w_cnt += 1
        if w_cnt >= num_to_predict:
          break

    target_labels = [self.tokenizer.vocab[x] if x else 0 for x in target_labels]
    return tokens, target_labels

  def _choice(self, rng, data, p):
    cul = np.cumsum(p)
    x = rng.random()*cul[-1]
    id = bisect(cul, x)
    return data[id]

  def _mask_token(self, idx, tokens, rng, mask_prob, keep_prob):
    label = tokens[idx]
    mask = '[MASK]'
    rand = rng.random()
    if rand < mask_prob:
      new_label = mask
    elif rand < mask_prob+keep_prob:
      new_label = label
    else:
      new_label = rng.choice(self.vocab_words)

    tokens[idx] = new_label

    return label

In [14]:
model_mlm.deberta.encoder.layer[0].intermediate.dense.weight

NameError: name 'logits' is not defined

In [86]:
mask_gen = NGramMaskGenerator(tokenizer, max_gram=1)

In [87]:
import numpy as np
import random
tokenizer.convert_tokens_to_ids(mask_gen.mask_tokens(tokens, rng=random)[0])

[32826, 16, 50264, 646, 32804, 530, 742, 9, 1470]

In [96]:
tokenizer.decode(mask_gen.mask_tokens(tokens, rng=random)[0])

'Paris FergusonK] of France'

In [104]:
tokenizer.mask_token_id

AttributeError: 'GPT2Tokenizer' object has no attribute 'mask_token_id'

In [98]:
mask_gen.mask_tokens(tokens, rng=random)[0]

['40313', '[MASK]', '262', '685', '31180', '42', '60', '286', '4881']

In [15]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
import numpy as np
from huggingface_hub import hf_hub_download
from DeBERTa.DeBERTa.apps.models import masked_language_model as deberta_mlm
from DeBERTa.DeBERTa.deberta import config


def _prepare_deberta(hf_model_name):
    config_json = hf_hub_download(hf_model_name, "config.json")
    d_config = config.ModelConfig.from_json_file(config_json)
    d_config.position_biased_input = True
    model = deberta_mlm.MaskedLanguageModel(d_config)
    checkpoint_path = hf_hub_download(hf_model_name, "pytorch_model.bin")
    model.load_state_dict(torch.load(checkpoint_path), strict=False)
    return model

deberta = _prepare_deberta("microsoft/deberta-v3-xsmall")

In [16]:
deberta

MaskedLanguageModel(
  (deberta): DeBERTa(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128100, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=384, out_features=384, bias=True)
              (key_proj): Linear(in_features=384, out_features=384, bias=True)
              (value_proj): Linear(in_features=384, out_features=384, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-07, elementwise_affi

In [58]:
hf_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall")
inputs = hf_tokenizer("Hello world", return_tensors='pt', padding=True)

logits = deberta(
                input_ids=inputs.input_ids.repeat(4, 1),
                labels=torch.diag(inputs.input_ids.squeeze()),
                input_mask=inputs.attention_mask,
                attention_mask=inputs.attention_mask,
            )["logits"]

/nlp/scr/ananthag/miniconda3/envs/syntax/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [61]:
logits[torch.arange(4), inputs.input_ids.squeeze()]

tensor([ 4.0100, -2.3126,  3.0641,  1.0293], grad_fn=<IndexBackward0>)

In [296]:
def _prepare_deberta(hf_model_name):
    checkpoint_path = hf_hub_download(hf_model_name, "pytorch_model.bin")
    ckpt = torch.load(checkpoint_path)
    model = AutoModelForMaskedLM.from_pretrained(hf_model_name)
    ckpt["cls.predictions.transform.dense.weight"] = ckpt.pop(
        "lm_predictions.lm_head.dense.weight"
    )
    ckpt["cls.predictions.transform.dense.bias"] = ckpt.pop(
        "lm_predictions.lm_head.dense.bias"
    )
    ckpt["cls.predictions.transform.LayerNorm.weight"] = ckpt.pop(
        "lm_predictions.lm_head.LayerNorm.weight"
    )
    ckpt["cls.predictions.transform.LayerNorm.bias"] = ckpt.pop(
        "lm_predictions.lm_head.LayerNorm.bias"
    )
    ckpt["cls.predictions.decoder.weight"] = ckpt[
        "deberta.embeddings.word_embeddings.weight"
    ]
    ckpt["cls.predictions.decoder.bias"] = ckpt["lm_predictions.lm_head.bias"]
    ckpt["cls.predictions.bias"] = ckpt.pop("lm_predictions.lm_head.bias")
    model.load_state_dict(ckpt, strict=False)
    return model

deberta = _prepare_deberta("microsoft/deberta-v3-base")

Some weights of DebertaV2ForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model = AutoModelForMaskedLM.from_pretrained("microsoft/deberta-v3-xsmall")
model

Some weights of DebertaV2ForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForMaskedLM(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 384, padding_idx=0)
      (LayerNorm): LayerNorm((384,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=384, out_features=384, bias=True)
              (key_proj): Linear(in_features=384, out_features=384, bias=True)
              (value_proj): Linear(in_features=384, out_features=384, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-07, elementwise_affine=True)
       

In [29]:
good = "Who have most people discovered while embarrassing Erin?"
bad = "Who have most people discovered Erin while embarrassing?"

In [30]:
import torch.nn.functional as F
def _encoder_log_prob_sum(lm, tokenizer, input_text_batch):
    assert len(input_text_batch) == 1
    tokenized = tokenizer(input_text_batch, return_tensors="pt")
    input_ids = tokenized.input_ids
    print(input_ids)
    attention_mask = tokenized.attention_mask
    seq_length = input_ids.shape[1]

    # Mask out each token in the input sequence
    masked_input_ids = input_ids.repeat(seq_length, 1)
    masked_attention_mask = attention_mask.repeat(seq_length, 1)
    masked_input_ids.fill_diagonal_(tokenizer.mask_token_id)

    labels = torch.diag(input_ids.squeeze())

    # Get the model's predictions for the batched masked inputs
    with torch.inference_mode():
        if isinstance(lm, deberta_mlm.MaskedLanguageModel):
            logits = lm(
                input_ids=masked_input_ids,
                labels=labels,
                input_mask=masked_attention_mask,
                attention_mask=masked_attention_mask,
            )["logits"]
            log_probs = F.log_softmax(logits, dim=-1)
            token_log_probs = log_probs[torch.arange(seq_length), input_ids.squeeze()]
        else:
            logits = lm(masked_input_ids, attention_mask=masked_attention_mask).logits

            log_probs = F.log_softmax(logits, dim=-1)
            token_log_probs = log_probs[
                torch.arange(seq_length), torch.arange(seq_length), input_ids.squeeze()
            ]
            print(token_log_probs)
    return token_log_probs.sum()

from transformers import AutoModelForMaskedLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall")
_encoder_log_prob_sum(deberta, tokenizer, [good])

tensor([[    1,  1876,   286,   370,   355,  2534,   438, 13784, 13736,   302,
             2]])
tensor([-20.0942, -21.9367, -21.6412, -18.3745, -18.6219, -15.2118, -16.8572,
        -15.4021, -17.8441, -23.4265, -17.6024])


tensor(-207.0126)

In [31]:
_encoder_log_prob_sum(deberta, AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall"), [bad])

tensor([[    1,  1876,   286,   370,   355,  2534, 13736,   438, 13784,   302,
             2]])
tensor([-18.9079, -20.7022, -23.6953, -18.6025, -19.8663, -13.3951, -17.7276,
        -18.0960, -12.1095, -23.7737, -18.3432])


tensor(-205.2192)

In [21]:
tokenizer("shocking")

{'input_ids': [1, 11081, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [32]:
deberta.dec

DebertaV2ForMaskedLM(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 384, padding_idx=0)
      (LayerNorm): LayerNorm((384,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=384, out_features=384, bias=True)
              (key_proj): Linear(in_features=384, out_features=384, bias=True)
              (value_proj): Linear(in_features=384, out_features=384, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-07, elementwise_affine=True)
       

In [289]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/Roberta-base")
deberta_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall")
tokens = deberta_tokenizer("One Two Three [MASK]", return_tensors="pt")
model = AutoModelForMaskedLM.from_pretrained("FacebookAI/Roberta-base")
mask_token_index = torch.where(tokens["input_ids"] == deberta_tokenizer.mask_token_id)[1]

In [290]:
tokens, mask_token_index

({'input_ids': tensor([[     1,    593,   1852,   3216, 128000,      2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])},
 tensor([4]))

In [294]:
logits = deberta(**tokens, position_ids=torch.arange(6).unsqueeze(0)).logits

In [295]:
deberta_tokenizer.batch_decode(torch.topk(logits[0][mask_token_index], 10).indices)

['BasqueEIS riveting Bayer💀 Maronite kawaii Akhtar Zahid繊']

In [56]:
logits.argmax(-1)

tensor([[    0, 32826,    16,     5,   646, 32804,   530,   742,     9,  1470,
             2]])

In [73]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [75]:
model.lm_head.decoder.bias

Parameter containing:
tensor([-0.0972, -0.0294,  0.4988,  ..., -0.0312, -0.0312, -1.0000],
       requires_grad=True)

In [76]:
deberta.cls.predictions.bias

Parameter containing:
tensor([0., 0., 0.,  ..., 0., 0., 0.], requires_grad=True)

In [78]:
checkpoint_path = hf_hub_download("microsoft/deberta-v3-xsmall", "pytorch_model.bin")
ckpt = torch.load(checkpoint_path)

In [81]:
ckpt['deberta.encoder.layer.0.attention.output.LayerNorm.bias']

tensor([-6.0730e-02,  3.3051e-02, -4.5532e-02, -7.3730e-02, -1.3245e-01,
        -1.0748e-01, -1.1218e-01, -2.1167e-01, -1.2408e-01, -2.1509e-01,
        -1.1212e-01, -1.4417e-01, -4.0375e-02, -4.4983e-02, -3.1677e-02,
        -3.6774e-02, -7.4829e-02, -2.3041e-02, -2.7393e-01, -1.7432e-01,
        -8.3679e-02, -9.1187e-02, -2.8152e-02, -2.7295e-01, -2.2278e-01,
        -1.0480e-01, -1.5588e-01, -8.8745e-02, -1.9409e-01, -1.3452e-01,
        -1.1774e-01, -3.4912e-02, -9.8511e-02, -1.9104e-01, -9.3933e-02,
         6.3538e-02, -4.8920e-02, -6.9427e-03, -1.1115e-01, -1.1072e-01,
        -9.4849e-02, -1.1981e-01, -6.7810e-02, -8.1238e-02, -1.3269e-01,
        -5.5328e-02, -3.5248e-02, -3.6774e-02, -1.0187e-01, -2.0850e-01,
        -1.2311e-01, -1.3208e-01, -7.9895e-02, -2.7417e-01, -1.2520e+00,
        -1.6370e-01, -4.3671e-02, -1.2903e-01, -7.5195e-02,  7.1533e-02,
        -1.0803e-01, -1.0645e-01, -1.1572e-01, -3.8391e-02, -1.4062e-01,
        -1.0583e-01, -1.6858e-01, -9.3506e-02, -1.2

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/Roberta-base")
model = AutoModelForMaskedLM.from_pretrained("FacebookAI/Roberta-base")

text = "The capital of France is Paris"
tokens = tokenizer(text, return_tensors="pt")
seq_length = tokens.input_ids.shape[1]
input_ids = tokens.input_ids.repeat(seq_length, 1)
input_ids.fill_diagonal_(tokenizer.mask_token_id)
attention_mask = tokens.attention_mask.repeat(seq_length, 1)
attention_mask = torch.tril(attention_mask)
input_ids, attention_mask

(tensor([[50264,   133,   812,     9,  1470,    16,  2201,     2],
         [    0, 50264,   812,     9,  1470,    16,  2201,     2],
         [    0,   133, 50264,     9,  1470,    16,  2201,     2],
         [    0,   133,   812, 50264,  1470,    16,  2201,     2],
         [    0,   133,   812,     9, 50264,    16,  2201,     2],
         [    0,   133,   812,     9,  1470, 50264,  2201,     2],
         [    0,   133,   812,     9,  1470,    16, 50264,     2],
         [    0,   133,   812,     9,  1470,    16,  2201, 50264]]),
 tensor([[1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1, 1, 1, 1]]))

In [5]:
masked_lm_output = model(input_ids, attention_mask=attention_mask, output_attentions=True)

In [14]:
masked_lm_output.attentions[1][4][0].half()

tensor([[9.8779e-01, 1.0853e-03, 1.0204e-03, 3.4695e-03, 6.5193e-03, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.5562e-01, 5.4199e-02, 4.0479e-01, 2.0886e-01, 7.6538e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.7441e-01, 2.2751e-02, 1.0773e-01, 1.4917e-01, 4.4604e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.3252e-01, 9.1476e-03, 7.3608e-02, 7.5256e-02, 5.0977e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [7.9541e-01, 1.2909e-02, 3.8208e-02, 2.0874e-02, 1.3257e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [9.2041e-01, 3.0655e-02, 2.0493e-02, 1.9007e-03, 2.6398e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [9.7168e-01, 2.0691e-02, 4.5090e-03, 7.6103e-04, 2.3079e-03, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [9.2334e-01, 4.6997e-02, 2.6260e-02, 1.3180e-03, 2.0828e-03, 0.0000e+00,
         0.0000e+00, 0.0000e+00]], dtype=torch.float16,
       grad_fn=<ToCopyBackward0>)

In [25]:
from transformers import AutoTokenizer, ElectraForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/electra-small-generator")
model = ElectraForMaskedLM.from_pretrained("google/electra-small-generator")

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].topk(3, axis=-1)
tokenizer.batch_decode(predicted_token_id.indices)

# labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# # mask labels of non-[MASK] tokens
# labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

# outputs = model(**inputs, labels=labels)
# round(outputs.loss.item(), 2)

['paris france bordeaux']

In [26]:
from transformers import AutoTokenizer, ElectraForCausalLM, ElectraConfig
import torch

tokenizer = AutoTokenizer.from_pretrained("google/electra-base-generator")
config = ElectraConfig.from_pretrained("google/electra-base-generator")
config.is_decoder = True
model = ElectraForCausalLM.from_pretrained("google/electra-base-generator", config=config)

inputs = tokenizer("The capital of France is", return_tensors="pt")
outputs = model(**inputs)

prediction_logits = outputs.logits

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/nlp/scr/ananthag/miniconda3/envs/syntax/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

In [30]:
tokenizer.batch_decode(prediction_logits.argmax(-1))

['. the capital of france is.']